In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

base_url = "https://books.toscrape.com/catalogue/page-{}.html"
books = []

for page in range(1, 51):  # 50 pages
    url = base_url.format(page)
    res = requests.get(url)
    if res.status_code != 200:
        break
    soup = BeautifulSoup(res.text, "html.parser")
    items = soup.find_all("article", class_="product_pod")

    for item in items:
        title = item.h3.a["title"]
        price = item.find("p", class_="price_color").text.strip()
        availability = item.find("p", class_="instock availability").text.strip()
        star_class = item.p["class"][1]  # e.g. "Three"
        books.append([title, price, availability, star_class])

df_books = pd.DataFrame(books, columns=["Title", "Price", "Availability", "Star Rating"])
df_books.to_csv("books.csv", index=False)
df_books.head()


,Title,Price,Availability,Star Rating
0,A Light in the Attic,Â£51.77,In stock,Three
1,Tipping the Velvet,Â£53.74,In stock,One
2,Soumission,Â£50.10,In stock,One
3,Sharp Objects,Â£47.82,In stock,Four
4,Sapiens: A Brief History of Humankind,Â£54.23,In stock,Five


In [6]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import random
import tempfile

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument(f"--user-data-dir={tempfile.mkdtemp()}")
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36")
options.add_experimental_option("excludeSwitches", ["enable-automation"])

driver = webdriver.Chrome(options=options)
driver.get("https://www.imdb.com/chart/top/")
time.sleep(random.uniform(3, 5))

soup = BeautifulSoup(driver.page_source, "html.parser")
driver.quit()

movies = []
for rank, row in enumerate(soup.select("li.ipc-metadata-list-summary-item")[:250], 1):
    title_elem = row.select_one("h3.ipc-title__text")
    title = title_elem.text.split('.', 1)[1].strip() if title_elem and '.' in title_elem.text else "N/A"
    year_elem = row.select_one("span.sc-b0691f29-8, span.cli-title-metadata-item")
    year = year_elem.text.strip("()") if year_elem else "N/A"
    rating_elem = row.select_one("span.ipc-rating-star--rating")
    rating = rating_elem.text if rating_elem else "N/A"
    movies.append([rank, title, year, rating])

pd.DataFrame(movies, columns=["Rank", "Title", "Year", "Rating"]).to_csv("imdb_top250.csv", index=False)

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}

time.sleep(random.uniform(1, 2))
response = requests.get('https://www.timeanddate.com/weather/', headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

cities = []
for table in soup.find_all('table'):
    for row in table.select('tbody tr'):
        city_tag = row.find('a')
        if city_tag:
            tds = row.find_all('td')
            if len(tds) >= 3:
                cities.append([city_tag.text.strip(), tds[1].text.strip(), tds[2].text.strip()])

pd.DataFrame(cities, columns=['City', 'Temperature', 'Condition']).to_csv('weather.csv', index=False)